In [47]:
import json
with open("main_json.json", "r", encoding="utf-8") as file:
    main_js = json.load(file)
print(len(main_js))

list_js = list(main_js)
for i in range(len(list_js)):
    elem = list(list_js[i].values())[0:3]
    print(elem)

143
['Точка невозврата', 'Вальтер Макс', 'Дементьев Илья']
['Путь Восходящего Солнца', 'Астахов Евгений,Булл Сергей', 'Кривошеев Вадим']
['Я вернулся. Том 6', 'Ткачев Андрей,Байяр А.', 'Федоров Кирилл']
['Возвышение клана', 'Шапочкин Александр,Широков Алексей', 'Радман Макс']
['Война клана', 'Шапочкин Александр,Широков Алексей', 'Радман Макс']
['Император Пограничья 3', 'Астахов Евгений,Токсик Саша', 'Дементьев Илья']
['Наследник поневоле', 'Гоблин,Каин', 'Ксеноморф']
['Пламя у границ империи', 'Бадевский Ян', 'Волков Роман']
['Каждому своё 3', 'Тармашев Сергей', 'Анашкин Александр,Арбузов Юрий,Буланова Наталья,Грядкин Григорий,Губарев Николай,Дмитриев Юрий,Егоров Алексей,Ефимовский Александр,Ефремова Екатерина,Жилин Николай,Жирякова Ирина,Журавлёв Артём,Панин Иван,Казакова Виолетта,Калашников Феликс,Касванд Герда,Колесников Даниил,Копп Олег,Котов Александр,Крилевич Сергей,Крылов Никита,Куренков Михаил,Ерастов Алексей,Малыгин Владимир,Меженин Геннадий,Микитюк Ева,Микитюк Евгений,Митроф

In [ ]:

import pandas as pd
import json
import pprint
df = pd.read_json("book_data.json")
row = list(df.iloc[-1][0:5])

row_len = len(df)

    
    
    
with open("main_json.json", "r", encoding="utf-8") as file:
    main_js = json.load(file)
print(len(main_js))

with open("book_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

dict_js = list(data[-1].values())


ls = []
for i in range(len(main_js)):
    item = list(main_js[i].values())[0:3]
    if set(item) == set(dict_js[0:3]):
        break
    ls.append(main_js[i])
        # print(item)
        # print(len(main_js))
    # print(i)
print(dict_js[0:3])
print(ls[-1])
print(len(ls))
# row = list(df.iloc[-1])
# # result = sorted(dict_js[0:5]) == sorted(row[0:5])
# result = set(dict_js[0:5]) == set(row[0:5])
# print(result)
# print(dict_js)
# print(row)



# print(set(dict_js[0:5]) == set(row))
# print(l)

# for i in range(len(df)):
#     r = df.iloc[i]
#     # print(r)
#     if set(r[0:3]) == set(list(data[-1].values())[0:3]):
#         print(r)
# list_1 = ['Темный мир Тараумара', 'Панченко Сергей', 'Ященко Игорь']
# list_2 = ['Темный мир Тараумара', 'Панченко Сергей', 'Ященко Игорь']

143
['Темный мир Тараумара', 'Панченко Сергей', 'Ященко Игорь']
{'Название': 'Некромант-1: в чужом мире', 'Автор': 'Винтеркей Влад', 'Читает': 'Кучерявых Ольга', 'Длительность': '06:14:27', 'Цикл': 'Сергей Громов(1)', 'Жанр': 'Фан-фэн/Попаданцы', 'Добавлена': ' 04.09.25', 'likes': '4', 'dis': '21'}
31


In [ ]:
import gspread

def get_first_cell_from_google_sheet(sheet_name: str, worksheet_name: str = "sheet1") -> str:
    """
    Функция получает первый элемент (ячейку A1) из Google таблицы.

    Аргументы:
        sheet_name (str): Название Google таблицы.
        worksheet_name (str): Название листа (по умолчанию "Sheet1").

    Возвращает:
        str: Значение ячейки A1.
    """
    # Подключение через сервисный аккаунт (ключи в JSON файле)
    gc = gspread.service_account(filename="config_gkey.json")

    # Открываем таблицу по имени
    sh = gc.open(sheet_name)

    # Открываем конкретный лист
    worksheet = sh.worksheet(worksheet_name)

    # Берём первый элемент (ячейку A1)
    first_value = worksheet.cell(1, 1).value

    print(first_value)
    return first_value


In [ ]:
import pandas as pd


def compare_google_and_dataframe(sheet_name: str, worksheet_name: str, df: pd.DataFrame, column_name: str):
    """
    Функция сравнивает данные из Google таблицы и Pandas DataFrame.

    Аргументы:
        sheet_name (str): Название Google таблицы.
        worksheet_name (str): Название листа в Google таблице.
        df (pd.DataFrame): Pandas DataFrame с данными.
        column_name (str): Название столбца DataFrame для сравнения.

    Возвращает:
        dict: Словарь с результатами сравнения:
              - 'only_in_google': значения только в Google таблице
              - 'only_in_dataframe': значения только в DataFrame
              - 'in_both': пересечение (общие значения)
    """
    # Подключение через сервисный аккаунт
    gc = gspread.service_account(filename="config_gkey.json")
    sh = gc.open(sheet_name)
    worksheet = sh.worksheet(worksheet_name)

    # Получаем все значения первого столбца Google Таблицы
    google_data = worksheet.col_values(1)  # список значений
    google_set = set(google_data)

    # Получаем данные из DataFrame
    dataframe_set = set(df[column_name].astype(str))  # приводим к строкам для корректного сравнения

    # Сравнение
    only_in_google = google_set - dataframe_set
    only_in_dataframe = dataframe_set - google_set
    in_both = google_set & dataframe_set

    return {
        "only_in_google": list(only_in_google),
        "only_in_dataframe": list(only_in_dataframe),
        "in_both": list(in_both),
    }

In [ ]:
# Пример DataFrame
data = {
    "names": ["Alice", "Bob", "Charlie", "Diana"]
}
df = pd.DataFrame(data)

# Получаем первый элемент из Google Таблицы
first_value = get_first_cell_from_google_sheet("config_gkey.json", "Sheet1")
print("Первый элемент:", first_value)

# Сравниваем данные
result = compare_google_and_dataframe("config_gkey.json", "Sheet1", df, "names")

print("Только в Google:", result["only_in_google"])
print("Только в DataFrame:", result["only_in_dataframe"])
print("Общие значения:", result["in_both"])


In [1]:
# datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
from datetime import datetime
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
import re
# GET -Получить данные со страници.
# POST -Передать данные или авторизацию.

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
page = 1
max_page = 1
data = []

while page <= max_page:

    link = (f'https://baza-knig.rip/fantastika-fenteze/page/{page}/')

    response = requests.get(link, headers=header)
    response.encoding = 'utf-8'  # Устанавливаем правильную кодировку

    soup = BeautifulSoup(response.text, 'html.parser')
    block = soup.find('div', id="dle-content")


    # Ищем все элементы с книгами
    books = block.find_all('div', class_='short')


    for book in books:
        book_data = {}

        # Извлекаем заголовок книги
        title = book.find('div', class_='short-title').text.strip()
        book_data["Название"] = title

        # Обрабатываем мета-данные книги
        for li in book.find('ul', class_='reset short-items').find_all('li'):
            parts = li.text.strip().split(':', 1) # Делим по первому вхождению ":
            if len(parts) == 2:
                key, value = parts
                book_data[key] = value

        data.append(book_data)
    page += 1


    # Эта конструкция помодет избавиться о Многоточей в страницах. И продолжеть Нумерацию страниц.
    button_navigation = soup.find('div', class_='bottom-nav ignore-select')
    pages = [p.text.strip() for p in button_navigation.find('div', class_="navigation")]
    int_pages = []
    for p in pages:
        try:
            n = int(p)
            int_pages.append(n)
        except:
            continue


# Удаление ненужныз символов.
patterns = {
    r"Фантастика": "Фан",
    r"фэнтези": "фэн",
    r",": "/",
    r" ": ""
}

for item in data:
    genre = item["Жанр"]
    for pattern, replacement in patterns.items():
        genre = re.sub(pattern, replacement, genre, flags=re.IGNORECASE)
    item["Жанр"] = genre




# df = pd.DataFrame(data)
df = pd.DataFrame(data).fillna("нет данных")
df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# df.head()


# print(len(data))


In [ ]:
gc = gspread.service_account(filename='.json')
wks = gc.open("web_parsing_baza_knig").sheet1
wks.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1b3GmBL5gA5px_BdnN37sSslnGlQgrnDqX-jiQTtKSVI',
 'updatedRange': "'Лист1'!A1:H12",
 'updatedRows': 12,
 'updatedColumns': 8,
 'updatedCells': 96}

In [10]:
status_kicke = []
for item in block.find_all('div', class_="short-bottom"):
    like_dislike = {}
    res = item.text.strip().split("\n")
    # like_dislike['like-dis-comm'] = f'like = {res[1]}/dis = {res[2]}/comment = {res[3]}'
    like_dislike['like-dis-comm'] = f'like = {res[1]}/dis = {res[2]}/comment = {res[3]}'
    status_kicke.append(like_dislike)

# log["print"] =
# df[status_kicke[0].keys()] = status_kicke
# df

[dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm']),
 dict_keys(['like-dis-comm'])]

In [ ]:
import gspread

gc = gspread.service_account(filename='.json')

# Open a sheet from a spreadsheet in one go
wks = gc.open("web_parsing_baza_knig").sheet1

# Update a range of cells using the top left corner address
wks.update([[1, 2], [3, 4]], 'A1')

# Or update a single cell
wks.update_acell('B42', "it's down there somewhere, let me take another look.")

# Format the header
wks.format('A1:B1', {'textFormat': {'bold': True}})

{'spreadsheetId': '1b3GmBL5gA5px_BdnN37sSslnGlQgrnDqX-jiQTtKSVI',
 'replies': [{}]}